In [141]:
import pandas as pd
import numpy as np
from optimization_engine import datathon_optimization_co2

In [142]:
prediction_dataset = 'data/processed/' + 'RF_predictions.csv'
predictions = pd.read_csv(prediction_dataset)

In [143]:
predictions.rename(columns={'date':'datetime','0':'solar_gen'}, inplace=True)
predictions['datetime'] = pd.to_datetime(predictions['datetime'])
predictions.set_index('datetime', inplace=True)
print(predictions.shape)
predictions.head()

(720, 1)


,solar_gen
datetime,
2024-09-01 00:00:00+02:00,0.057787
2024-09-01 01:00:00+02:00,0.029787
2024-09-01 02:00:00+02:00,0.000400
2024-09-01 03:00:00+02:00,0.004135
2024-09-01 04:00:00+02:00,0.004371


In [144]:
consumption = pd.read_csv('data/raw/consumo_sept.csv')
consumption.rename(columns={'FECHA': 'datetime', 'TOTAL_KWH_ENERGIA':'consumption'}, inplace=True)
consumption['datetime'] = pd.to_datetime(consumption['datetime'])
consumption.set_index('datetime', inplace=True)
consumption.index = consumption.index.tz_localize('Europe/Madrid', ambiguous='NaT', nonexistent='shift_forward')
print(consumption.shape)
consumption.head()

(720, 1)


,consumption
datetime,
2024-09-01 00:00:00+02:00,110.855888
2024-09-01 01:00:00+02:00,107.341202
2024-09-01 02:00:00+02:00,105.550308
2024-09-01 03:00:00+02:00,106.840103
2024-09-01 04:00:00+02:00,105.879791


In [145]:
co2 = pd.read_csv('data/raw/ES_2024_hourly.csv')
co2 = co2[['Datetime (UTC)', 'Carbon Intensity gCO₂eq/kWh (direct)']]
co2.rename(columns={'Datetime (UTC)': 'datetime', 'Carbon Intensity gCO₂eq/kWh (direct)': 'co2_emissions'}, inplace=True)
co2['datetime'] = pd.to_datetime(co2['datetime'], utc=True )
co2.set_index('datetime', inplace=True)
co2.index = co2.index.tz_convert('Europe/Madrid')
co2.head()

,co2_emissions
datetime,
2024-01-01 01:00:00+01:00,93.63
2024-01-01 02:00:00+01:00,93.64
2024-01-01 03:00:00+01:00,98.90
2024-01-01 04:00:00+01:00,99.33
2024-01-01 05:00:00+01:00,102.59


In [146]:
df = predictions.join(consumption, how='inner').join(co2, how='inner')
df.index = df.index.tz_convert('Europe/Madrid')
print(df.shape)

(720, 3)


In [150]:
hrs = 24
days = df.shape[0] // hrs
assert df.shape[0] / hrs == df.shape[0] // hrs, 'The dataset doesn\'t contain full day cycles'

x = np.array([])
print(x)

for day in range(days):
    start_index = day * 24
    end_index = (day + 1) * 24
    day_df = df.iloc[start_index:end_index]
    c = day_df['consumption'].to_numpy()
    s = day_df['solar_gen'].to_numpy()
    co2 = day_df['co2_emissions'].to_numpy()
    solution = datathon_optimization_co2(S=s,C=c,CO2=co2)
    x = np.append(x, solution.x)

x.shape

[]
The Naive Guess for this iteration is [ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  5.05260369  0.          0.          0.          0.         -5.05260369]
[{'type': 'ineq', 'fun': <function datathon_optimization_co2.<locals>.surplus_constraint at 0x000002139859B100>}, {'type': 'ineq', 'fun': <function datathon_optimization_co2.<locals>.soc_constraint at 0x0000021398598360>}, {'type': 'ineq', 'fun': <function datathon_optimization_co2.<locals>.cycle_constraint at 0x000002139859B240>}]
Optimization terminated successfully    (Exit mode 0)
            Current function value: -4308.240976838065
            Iterations: 6
            Function evaluations: 50
            Gradient evaluations: 2
The Naive Guess for this iteration is [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0. -0.]
[{'type': '

(720,)

In [151]:
df['battery_charge'] = x.round(4)
df['solar_usage'] = df.apply(lambda row: min(row['solar_gen'], row['consumption']), axis=1)
df['battery_usage'] = df.apply(lambda row: abs(row['battery_charge'] if row['battery_charge'] < 0 else 0), axis=1)
df['carbon_reduction'] = df.apply(lambda row: row['co2_emissions'] * row['battery_usage'] ,axis=1)
df['false_results'] = df.apply(lambda row: row['battery_charge'] > row['solar_surplus'],axis=1)
df.head(50)

,solar_gen,consumption,co2_emissions,solar_surplus,solar_surplus_cumsum,hour,day,Naive_x0,battery_charge,solar_usage,battery_usage,carbon_reduction,false_results
datetime,,,,,,,,,,,,,
2024-09-01 00:00:00+02:00,0.057787,110.855888,158.77,0.000000,0.000000,0,1,0.000000,-0.0000,0.057787,0.0000,0.000000,False
2024-09-01 01:00:00+02:00,0.029787,107.341202,164.81,0.000000,0.000000,1,1,0.000000,0.0000,0.029787,0.0000,0.000000,False
2024-09-01 02:00:00+02:00,0.000400,105.550308,170.04,0.000000,0.000000,2,1,0.000000,-0.0000,0.000400,0.0000,0.000000,False
2024-09-01 03:00:00+02:00,0.004135,106.840103,175.13,0.000000,0.000000,3,1,0.000000,-0.0000,0.004135,0.0000,0.000000,False
2024-09-01 04:00:00+02:00,0.004371,105.879791,175.53,0.000000,0.000000,4,1,0.000000,0.0000,0.004371,0.0000,0.000000,False
2024-09-01 05:00:00+02:00,0.004371,106.000397,174.22,0.000000,0.000000,5,1,0.000000,-0.0000,0.004371,0.0000,0.000000,False
2024-09-01 06:00:00+02:00,0.004371,105.497681,175.05,0.000000,0.000000,6,1,0.000000,0.0000,0.004371,0.0000,0.000000,False
2024-09-01 07:00:00+02:00,0.009861,101.122337,170.19,0.000000,0.000000,7,1,0.000000,-0.0000,0.009861,0.0000,0.000000,False
2024-09-01 08:00:00+02:00,0.489463,97.691238,159.95,0.000000,0.000000,8,1,0.000000,0.0000,0.489463,0.0000,0.000000,False


In [152]:
df.to_csv('data/processed/optimization-result.csv')

In [153]:
base_ra2 = sum(df['solar_usage']) / sum(df['solar_gen'])
ra2 = (sum(df['solar_usage']) + sum(df['battery_usage'])) / sum(df['solar_gen'])
print('Base Ra factor of the facility without battery is: ', base_ra2)
print('Optimize Ra factor of the facility with battery is: ', ra2)

Base Ra factor of the facility without battery is:  0.9417120194732587
Optimize Ra factor of the facility with battery is:  0.9811032153808784


In [154]:
sum(df['solar_gen'])

19692.331804781566

In [155]:
sum(df['solar_usage']) 

18544.50555201833

In [156]:
sum(df['battery_usage'])

775.7045